# Analysis of the Global Terrorism Database
https://www.start.umd.edu/gtd/

In [11]:
%matplotlib inline 
import numpy as np # imports a fast numerical programming library
import scipy as sp #imports stats functions, amongst other things
import matplotlib as mpl # this actually imports matplotlib
import matplotlib.cm as cm #allows us easy access to colormaps
import matplotlib.pyplot as plt #sets up plotting under plt
import pandas as pd #lets us handle data as dataframes
import seaborn as sns #sets up styles and gives us more plotting options

import statsmodels
import sklearn
import nltk

from collections import Counter
import string
import re

from datetime import datetime

#sets up pandas table display
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)


In [54]:
def load_data():

    file_ = "data/globalterrorismdb_0615dist.csv"
    
    #df = pd.read_csv(file_, header=0, nrows=100, dtype=str)
    df = pd.read_csv(file_, header=0, dtype=str)

    # fix dates
    df['iyear'] = df['iyear'].astype(int)
    df['imonth'] = df['imonth'].astype(int)
    df['iday'] = df['iday'].astype(int)

    df.ix[df.imonth == 0, 'imonth'] = 1
    df.ix[df.iday == 0, 'iday'] = 1

    df['date'] = df[['iyear', 'imonth', 'iday']].apply(lambda s : datetime(*s),axis = 1)
        
    print("Data loaded")

    return df

df = load_data()

Data loaded


In [61]:
# want to test if i'm reading all the data

# text_cols = ['summary','alternative_txt','attacktype1_txt','attacktype2_txt','attacktype3_txt','targtype1_txt','targsubtype1_txt','natlty1_txt','targtype2_txt','targsubtype2_txt','natlty2_txt','targtype3_txt','targsubtype3_txt','natlty3_txt','motive','guncertain1','guncertain2','guncertain3','nperps','nperpcap','claimmode_txt','claimmode2_txt','claimmode3_txt','compclaim','weaptype1_txt','weapsubtype1_txt','weaptype2_txt','weapsubtype2_txt','weaptype3_txt','weapsubtype3_txt','weaptype4_txt','weapsubtype4_txt','weapdetail','propextent_txt','propcomment','ishostkid','nhostkid','nhostkidus','divert','kidhijcountry','ransom','ransomnote','hostkidoutcome_txt','addnotes','scite1','scite2','scite3','related']

# df[text_cols] = df[text_cols].fillna('')

# df['full_text'] = df[text_cols].apply(lambda x: ' '.join(x), axis=1)

# df.full_text = df.full_text.apply(lambda x: regex.sub(' ', x.lower()).replace('  ', ' ').replace('  ', ' '))

# df.shape

# words = []
# word_count = 0

# for i in range(140000,140999):
    
#     for col in text_cols:
#         #df.loc[100][col]

#         test_summ = df[i:i+1][col].apply(lambda x: regex.sub(' ', x.lower()).replace('  ', ' ').replace('  ', ' '))

#         words = words + test_summ.tolist()[0].split()
        

# len(words)
# # 78936 words 0, 999
# # 97420 words 140000,140999

(141966, 136)

In [62]:
# text_cols = []
# for col in df.columns:
#     print(col, df[col].str.len().sum())
#     text_cols.append(col)

regex = re.compile('[^a-zA-Z\s]')

text_cols = ['summary','alternative_txt','attacktype1_txt','attacktype2_txt','attacktype3_txt','targtype1_txt','targsubtype1_txt','natlty1_txt','targtype2_txt','targsubtype2_txt','natlty2_txt','targtype3_txt','targsubtype3_txt','natlty3_txt','motive','guncertain1','guncertain2','guncertain3','nperps','nperpcap','claimmode_txt','claimmode2_txt','claimmode3_txt','compclaim','weaptype1_txt','weapsubtype1_txt','weaptype2_txt','weapsubtype2_txt','weaptype3_txt','weapsubtype3_txt','weaptype4_txt','weapsubtype4_txt','weapdetail','propextent_txt','propcomment','ishostkid','nhostkid','nhostkidus','divert','kidhijcountry','ransom','ransomnote','hostkidoutcome_txt','addnotes','scite1','scite2','scite3','related']

df[text_cols] = df[text_cols].fillna('')

df['full_text'] = df[text_cols].apply(lambda x: ' '.join(x), axis=1)

df.full_text = df.full_text.apply(lambda x: regex.sub(' ', x.lower()).replace('  ', ' ').replace('  ', ' '))


In [66]:
def word_frequency(s, col_name='word_frequency'):
    # takes a text string and returns a word frequency dataframe
    
    total_words = len(s)
    
    # Counter returns list of tuples
    word_counter = Counter(s.split()) #.most_common()
    
    df = pd.DataFrame.from_dict(word_counter, orient='index')
    df.rename(columns={0: col_name}, inplace=True)
    
    df[col_name] = df[col_name] / total_words
    
    return df

def word_tags(df):
    # takes a word frequency dataframe and adds a column for tags
    
    tags = nltk.pos_tag(df.index)
    df_tags = pd.DataFrame(tags, columns=['words', 'tag'])
    df_tags.set_index('words', inplace=True) #.index.name = None
    df_tags.index.name = None
    
    df = df.join(df_tags)
    
    return df


In [68]:
df_early = word_frequency(df_by_year[10:19].tolist()[0], 'early')
df_later = word_frequency(df_by_year[25:34].tolist()[0], 'later')

#print(len(df_by_year[10:19].words))

df_words = word_tags(df_early.join(df_later))
df_words['change'] = (df_words.later - df_words.early) / df_words.early
#df_words = df_words.set_index('words')


df_trends = df_words.sort_values('change', ascending=False)[0:20].append(df_words.sort_values('change')[0:10]).sort_values('change', ascending=False)

fig, axes = plt.subplots(nrows=1, ncols=1)
df_trends.change.plot(kind='bar', ax=axes, figsize=(12, 10), title='Change in Job Listing Words')
plt.savefig('word_frequency_change.png', bbox_inches='tight')

df_words.sort_values('change', ascending=False)
df_words

AttributeError: 'DataFrame' object has no attribute 'tolist'

In [67]:

#nltk.pos_tag(df_words.index)
text = 'And now for something completely different'

df1 = word_frequency(text)

tokens = text.split()
#nltk.pos_tag()
df1 #.index

,word_frequency
completely,0.02381
something,0.02381
different,0.02381
now,0.02381
for,0.02381
And,0.02381


In [5]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/steven/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True